In [1]:
%%capture
%pip install -U bitsandbytes
%pip install transformers==4.44.2
%pip install -U accelerate
%pip install -U peft
%pip install -U trl

In [2]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(token = hf_token)

In [3]:
base_model_url = "/kaggle/input/qwen-3/transformers/1.7b/1"
new_model_url = "/kaggle/input/qwen-finetune-test/qwen-3-1.7b-wealth-finetune"

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format

tokenizer = AutoTokenizer.from_pretrained(base_model_url)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model_url,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

2025-05-19 23:18:35.446373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747696715.639321      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747696715.692394      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
tokenizer.chat_template = None
base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)
model = PeftModel.from_pretrained(base_model_reload, new_model_url)

model = model.merge_and_unload()

In [6]:
instruction = """You are a wealth management assistant. 
     Be professional be matter of fact about your points.
    """

messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "What are common investing strategies?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150)

generated_tokens = outputs[0][inputs['input_ids'].shape[-1]:]

response = tokenizer.decode(generated_tokens, skip_special_tokens=True)

print(response.strip())

<think>

</think>

I'm happy to help! Investing is a great way to grow your wealth. Here are some common investing strategies:

1. Long-term investing: This strategy involves holding investments for a long period, often years or decades. It aims to benefit from compound growth and potential market appreciation.

2. Diversification: This strategy involves spreading investments across different asset classes, such as stocks, bonds, and real estate. Diversification helps reduce risk and increase the potential for returns.

3. Index investing: This strategy involves investing in a broad market index, such as the S&P 500 or the Dow Jones. It is a low-cost and passive approach to investing that tracks the overall performance of the market.

4. Active


In [7]:
new_model = "qwen-3-1.7b-wealth-finetune-merged"

model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('qwen-3-1.7b-wealth-finetune-merged/tokenizer_config.json',
 'qwen-3-1.7b-wealth-finetune-merged/special_tokens_map.json',
 'qwen-3-1.7b-wealth-finetune-merged/vocab.json',
 'qwen-3-1.7b-wealth-finetune-merged/merges.txt',
 'qwen-3-1.7b-wealth-finetune-merged/added_tokens.json',
 'qwen-3-1.7b-wealth-finetune-merged/tokenizer.json')

In [8]:
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/young43/qwen-3-1.7b-wealth-finetune-merged/commit/9fb89a4bb865b7667c9686b9317b2ca9b1bca865', commit_message='Upload tokenizer', commit_description='', oid='9fb89a4bb865b7667c9686b9317b2ca9b1bca865', pr_url=None, repo_url=RepoUrl('https://huggingface.co/young43/qwen-3-1.7b-wealth-finetune-merged', endpoint='https://huggingface.co', repo_type='model', repo_id='young43/qwen-3-1.7b-wealth-finetune-merged'), pr_revision=None, pr_num=None)